In [190]:
from bs4 import BeautifulSoup
import requests
import string
import re
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()


In [191]:
# all_disease_urls = []
# columns = ["disease_name","disease_url"]
# for current_disease_set_page in fetch_disease_pages_a_z:
#     soup = BeautifulSoup(current_disease_set_page)
#     disease_div = soup.find_all("div", attrs={'class': 'all-disease'})
#     disease_a = disease_div[0].contents[1].find_all("a")
#     for a_tags in disease_a:
#         all_disease_urls.append((a_tags.contents[1].text,f"https://www.nhp.gov.in{a_tags['href']}"))
# disease_link_DF = spark.createDataFrame(all_disease_urls).toDF(*columns)

In [192]:
my_disease_link_df = spark.read.option('header', True).csv("./my_file.csv") 

In [193]:
save_disease_pages = []

for current_disease in my_disease_link_df.collect():
    my_disease_category = re.sub(r'[^A-Za-z0-9 "{}:]+', ' ', current_disease["disease_url"].split("/")[4]).strip().title()
    save_disease_pages.append((requests.get(current_disease["disease_url"]).text, str(current_disease["disease_name"]), str(my_disease_category), str(current_disease["disease_url"])))

In [194]:
# # create dataframe 
# my_columns = ["page_string", "disease_name", "disease_category"]
# my_full_data_export = spark.createDataFrame(save_disease_pages).toDF(*my_columns)
# my_full_data_export.toPandas().to_csv("./export/full_load.csv", index=False)

In [205]:
import re
import bs4
import validators
def create_strings_introduction(current_element : list):
    my_reference_list = []
    my_string_list = []

    for element in current_element:
        
        if(len(element.contents)>1):
            for current_p in element.find_all('p'):
                for p_elements in current_p.contents:
                    if(type(p_elements) == bs4.element.NavigableString):
                        my_current_p_tag_contents = str(p_elements).strip()
                        if(len(my_current_p_tag_contents) > 2):
                            my_string_list.append(my_current_p_tag_contents.replace(u'\xa0', u' '))
            for current_a in element.find_all('a'):
                if current_a.get('href') !=None:
                    my_reference_list.append(current_a.get('href'))
    return  my_string_list, my_reference_list

In [206]:
def process_page_details(my_page_list):
    my_out_data = []
    for current_page_detail in my_page_list:
        if(type(current_page_detail) == bs4.element.Tag):
            my_out_data.append(
                (
                    str(current_page_detail.contents[0].contents[0]).title(), 
                    str(str(current_page_detail.contents[1])).replace(":","").strip()
                )
            )

    return my_out_data

    

In [207]:
def process_symptoms(symptom_list : list):
    my_reference_list = []
    my_symptom_list = []
    my_symptom_div = symptom_list[0].contents
    for div_elements in my_symptom_div:
        if(type(div_elements) == bs4.element.Tag):
            my_li = div_elements.find_all('li')
            my_links_colls = div_elements.find_all('a')
            if(len(my_li)>0):
                for li_tag in my_li:
                    for li_in_li in li_tag.contents:
                        if(type(li_in_li) == bs4.element.NavigableString):
                            my_processed_string = str(li_in_li).strip()
                            if(len(my_processed_string)>1):
                                my_symptom_list.append(my_processed_string)
                        elif(type(li_in_li) == bs4.element.Tag):
                            if(li_in_li!=None):
                                my_extra_a = li_in_li.get('href')
                                my_other_elem = li_in_li.contents
                                if(my_extra_a!=None):
                                    my_reference_list.append(my_extra_a)
                                else:
                                    for current_content in my_other_elem:
                                        if(type(current_content) == bs4.element.NavigableString):
                                            my_symptom_list.append(current_content)
            if(my_links_colls!=None):
                for current_a_tag in my_links_colls:
                    my_reference_list.append(current_a_tag.get('href'))
    return (my_reference_list, my_symptom_list)

In [208]:
def general_processor(cause_list : list):
    for cause in cause_list:
        my_parameter_list = []
        my_reference_list = []

        get_all_li = cause.find_all('li')
        get_all_strong = cause.find_all('strong')
        get_all_p = cause.find_all('p')
        get_all_a = cause.find_all('a')

        for current_li in get_all_li:
            for current_li_content in current_li.contents:
                if(type(current_li_content) == bs4.element.NavigableString):
                    if(len(str(current_li_content).strip())>2):
                        my_parameter_list.append(str(current_li_content).replace(':', '').strip().replace(u'\xa0', u' '))


        for current_strong in get_all_strong:
            for current_strong_content in current_strong.contents:
                if(type(current_strong_content) == bs4.element.NavigableString):
                    if(len(str(current_strong_content).strip())>2):
                        my_parameter_list.append(str(current_strong_content).replace(':', '').strip().replace(u'\xa0', u' '))

        for current_p in get_all_p:
            for current_p_content in current_p.contents:
                if(type(current_p_content) == bs4.element.NavigableString):
                    if(len(str(current_p_content).strip())>2):
                        my_parameter_list.append(str(current_p_content).replace(':', '').strip().replace(u'\xa0', u' '))

        for current_a in get_all_a:
            my_reference_list.append(current_a.get('href'))

    return my_parameter_list, my_reference_list

In [209]:
import pyodbc
from datetime import datetime
connection_string = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=DISEASE_WAREHOUSE;'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=yes;'
)
my_odbc_connection = pyodbc.connect(connection_string)
current_user = "MANUAL_LOAD"
my_odbc_cursor = my_odbc_connection.cursor()
def insert_disease_source(current_disease_source : str, current_disease_description : str, current_disease_source_link : str):
    my_source_id = None
    my_disease_source_check = f"""SELECT source_id from [dbo].[source_information] where [source_link] = ?"""
    my_odbc_cursor.execute(my_disease_source_check,(current_disease_source_link,))
    my_source_tuple = my_odbc_cursor.fetchone()
    if(my_source_tuple == None):
        my_disease_source_query = f"""INSERT INTO [dbo].[source_information]
            ([source_name]
            ,[source_description]
            ,[source_link]
            ,[created_by]
            ,[created_date]
            ,[updated_by]
            ,[updated_date])
        OUTPUT inserted.source_id
        VALUES
            (?
            ,?
            ,?
            ,'{current_user}'
            ,?
            ,'{current_user}'
            ,?);"""
        
        my_odbc_cursor.execute(
            my_disease_source_query,
            (current_disease_source, 
            current_disease_description, 
            current_disease_source_link, 
            datetime.now(), 
            datetime.now())
            )
        my_source_id = my_odbc_cursor.fetchone()[0]
        my_odbc_cursor.commit()
        return my_source_id
    else:
        my_odbc_cursor.commit()
        return my_source_tuple[0]
def insert_disease_category(current_disease_category : str, description : str):
    my_category_id = None
    my_query_check = f"""SELECT category_id FROM [dbo].[disease_category] WHERE [category_name] = ?"""
    my_odbc_cursor.execute(my_query_check, current_disease_category)
    my_tuple = my_odbc_cursor.fetchone()
    if(my_tuple == None):
        my_query = f"""INSERT INTO [dbo].[disease_category]
            ([category_name]
            ,[category_description]
            ,[created_by]
            ,[created_date]
            ,[updated_by]
            ,[updated_date])
        OUTPUT inserted.category_id
        VALUES
            (?
            ,?
            ,'{current_user}'
            ,?
            ,'{current_user}'
            ,?)"""
        my_odbc_cursor.execute(
            my_query,
            (current_disease_category, 
            description, 
            datetime.now(), 
            datetime.now())
            )
        my_category_id = my_odbc_cursor.fetchone()[0]
        my_odbc_cursor.commit()
        return my_category_id
    else:
        my_odbc_cursor.commit()
        return my_tuple[0]
def insert_disease(
        category_id : int, 
        source_id : int, 
        disease_name : str, 
        disease_description  :str,
        published_by : str,
        published_date : datetime,
        created_or_validated_by : str,
        last_updated_on : str):
    
    getdisease_id = None

    my_disease_master_query_check = f"""SELECT source_id FROM [dbo].[diseases_master] WHERE [disease_name] = ?"""
    my_odbc_cursor.execute(my_disease_master_query_check, disease_name)
    my_tuple = my_odbc_cursor.fetchone()
    if(my_tuple==None):
        my_disease_master_query = f"""INSERT INTO [dbo].[diseases_master]
            ([category_id]
            ,[source_id]
            ,[disease_name]
            ,[disease_description]
            ,[published_by]
            ,[published_date]
            ,[created_or_validated_by]
            ,[last_updated_on]
            ,[created_by]
            ,[created_date]
            ,[updated_by]
            ,[updated_date])
        OUTPUT inserted.disease_id
        VALUES
            (?
            ,?
            ,?
            ,?
            ,?
            ,?
            ,?
            ,?
            ,'{current_user}'
            ,?
            ,'{current_user}'
            ,?)"""
        
        my_odbc_cursor.execute(
            my_disease_master_query,
            (category_id, 
            source_id,
            disease_name, 
            disease_description,
            published_by, 
            published_date,
            created_or_validated_by,
            last_updated_on, 
            datetime.now(), 
            datetime.now())
            )
        getdisease_id = my_odbc_cursor.fetchone()[0]
        my_odbc_cursor.commit()
        return getdisease_id
    else:
        my_odbc_cursor.commit()
        return my_tuple[0]
def insert_reference(reference_string : str):
    get_reference_id = None
    check_if_exists = f"""SELECT reference_id FROM [dbo].[information_references] where [reference_name] = ?"""
    my_odbc_cursor.execute(check_if_exists, (reference_string,))
    existing_reference = my_odbc_cursor.fetchone()

    if(existing_reference==None):
        insert_reference = f"""INSERT INTO [dbo].[information_references]
               ([reference_type_id]
               ,[reference_name]
               ,[created_by]
               ,[created_date]
               ,[updated_by]
               ,[updated_date])
        OUTPUT inserted.reference_id
         VALUES
               (1
               ,?
               ,'{current_user}'
               ,?
               ,'{current_user}'
               ,?)"""
        my_odbc_cursor.execute(
            insert_reference,
            (reference_string, 
             datetime.now(), 
             datetime.now())
            )
        get_reference_id = my_odbc_cursor.fetchone()[0]
        my_odbc_cursor.commit()
        return get_reference_id
        
    else:
        my_odbc_cursor.commit()
        return existing_reference[0]
def create_disease_reference_relationship(disease_id, category_id, source_id, reference_id):

    my_tuple = None
    check_if_reference_exists = f"""SELECT disease_id, category_id, source_id, reference_id,disease_reference_id FROM [dbo].[disease_reference] WHERE reference_id = ?;"""
    my_odbc_cursor.execute(check_if_reference_exists, reference_id)
    my_tuple = my_odbc_cursor.fetchone()
    if(my_tuple==None):
        my_insert_query = f"""INSERT INTO [dbo].[disease_reference]
            ([disease_id]
            ,[category_id]
            ,[source_id]
            ,[reference_id]
            ,[created_by]
            ,[created_date]
            ,[updated_by]
            ,[updated_date])
        OUTPUT inserted.disease_id, inserted.category_id, inserted.source_id, inserted.reference_id, inserted.disease_reference_id
        VALUES
                (?
                ,?
                ,?
                ,?
                ,'{current_user}'
                ,?
                ,'{current_user}'
                ,?)"""
        
        my_odbc_cursor.execute(my_insert_query, (
            disease_id, category_id, source_id, reference_id, datetime.now(), datetime.now()))
        my_tuple = my_odbc_cursor.fetchone()
        my_odbc_cursor.commit()
        return my_tuple
    else:
        my_odbc_cursor.commit()
        return my_tuple
def insert_symptom(symptom_string : str, symptom_description : str):

    my_symptom_query_check = f"""SELECT symptom_id FROM [dbo].[symptoms] WHERE symptom_name = ?"""
    my_odbc_cursor.execute(my_symptom_query_check, symptom_string)
    my_tuple = my_odbc_cursor.fetchone()
    if(my_tuple == None):
        my_symptom_query = f"""INSERT INTO [dbo].[symptoms]
           ([symptom_name]
           ,[symptom_description]
           ,[created_by]
           ,[created_date]
           ,[updated_by]
           ,[updated_date])

    OUTPUT inserted.symptom_id
     VALUES
           (?
           ,?
           ,'{current_user}'
            ,?
            ,'{current_user}'
            ,?)"""

        my_odbc_cursor.execute(
            my_symptom_query, (symptom_string, symptom_description, datetime.now(), datetime.now()))
        return my_odbc_cursor.fetchone()[0]
    else:
        my_odbc_cursor.commit()
        return my_tuple[0]
def push_disease_details(
        detail_name : list, 
        detail_description  :str,
        disease_id : int,
        category_id  : int,
        source_id : int,
        detail_type_id : int
        ):
    check_disease_detail_query = f"""SELECT detail_id FROM [dbo].[details] 
    WHERE [detail_name] = ? AND [detail_description] = ?;"""
    my_odbc_cursor.execute(check_disease_detail_query, (detail_name, detail_description))
    my_tuple = my_odbc_cursor.fetchone()
    if(my_tuple == None):
        insert_disease_detail_query = f"""INSERT INTO [dbo].[details]
        ([detail_type_id]
        ,[detail_name]
        ,[detail_description]
        ,[created_by]
        ,[created_date]
        ,[updated_by]
        ,[updated_date])
        OUTPUT inserted.detail_id
        VALUES(?,?,?,'{current_user}',?,'{current_user}',?)"""
        my_odbc_cursor.execute(insert_disease_detail_query, (detail_type_id, detail_name, detail_description,datetime.now(),datetime.now()))
        return_detail_id = my_odbc_cursor.fetchone()[0]
        my_odbc_cursor.commit()
        insert_disease_relationship = f"""INSERT INTO [dbo].[disease_details]
           ([disease_id]
           ,[category_id]
           ,[source_id]
           ,[detail_id]
           ,[detail_type_id]
           ,[created_by]
           ,[created_date]
           ,[updated_by]
           ,[updated_date])
    OUTPUT inserted.disease_id, inserted.category_id, inserted.source_id, inserted.detail_id, inserted.detail_type_id
    VALUES
           (?
           ,?
           ,?
           ,?
           ,?
           ,'{current_user}',?,'{current_user}',?)"""
        my_odbc_cursor.execute(insert_disease_relationship, (
            disease_id,
            category_id,
            source_id,
            return_detail_id,
            detail_type_id,
            datetime.now(),
            datetime.now(),
            ))
        my_final_tup = my_odbc_cursor.fetchone()
        my_odbc_cursor.commit()
        return my_final_tup
    else:
        check_disease_detail_ref = f"""SELECT 
        disease_id, category_id, source_id, detail_id, detail_type_id FROM [dbo].[disease_details] WHERE detail_id = ?;"""
        my_odbc_cursor.execute(check_disease_detail_ref, (my_tuple[0],))
        my_final_tuple = my_odbc_cursor.fetchall()
        my_odbc_cursor.commit()
        return my_final_tuple



    
    

In [210]:
def create_disease_details(my_array,get_disease_id, get_category_id, get_source_id, description, detail_type):
    detail, detail_ref = general_processor(my_array)
    for current_detail in detail:
        push_disease_details(current_detail, description, get_disease_id, get_category_id, get_source_id, detail_type)
    for c_detail_ref in detail_ref:
        if(c_detail_ref==None):
            c_detail_ref = "Not available"
        reference_id = insert_reference(c_detail_ref)
        my_disease_relation = create_disease_reference_relationship(get_disease_id, get_category_id,get_source_id, reference_id)

In [212]:
for current_disease_webpage in save_disease_pages:
    page_details = BeautifulSoup(current_disease_webpage[0]).find_all("ul", attrs={'style': 'padding: 0; margin: 0; list-style-type: none;'})[0].contents
    Introduction_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Introduction'})
    Symptoms_arr =  BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Symptoms'})
    Causes_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Causes'})
    Diagnosis_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Diagnosis'})
    Management_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Management'})
    Complications_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Complications'})
    Prevention_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Prevention'})
    Classification_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Classification'})
    Transmission_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Transmission'})
    Treatment_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Treatment'})
    Cancer_Myths_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Cancer Myths'})
    symptomsvoice_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'symptomsvoice'})
    Diseasedescription_arr = BeautifulSoup(current_disease_webpage[0]).find_all("div", attrs={'id': 'Diseasedesciption'})

    get_source_id = insert_disease_source(
        "NHP", "National Health Portal of India, Gateway to Authentic Health Information", current_disease_webpage[3])
    
    get_category_id = insert_disease_category(str(current_disease_webpage[2]).strip(), current_disease_webpage[3])

    intro_list, ref_list_intro, = create_strings_introduction(Introduction_arr)
    my_page_details_list = process_page_details(page_details)

    try:
        publised_date = datetime.strptime(my_page_details_list[0][1], "%b %d, %Y")
        last_updated_on = datetime.strptime(my_page_details_list[3][1], "%b %d, %Y")
    except:
        publised_date = datetime.now()
        last_updated_on = datetime.now()

    try:
        published_by = my_page_details_list[1][1]
    except:
        published_by = "Not Avaialble"

    try:
        created_or_validated_by = my_page_details_list[2][1]
    except:
        created_or_validated_by = "Not Avaialble"
    get_disease_id = insert_disease(
        get_category_id, 
        get_source_id, 
        str(current_disease_webpage[1]).strip().title(),
        " ".join(intro_list),
        published_by,
        publised_date,
        created_or_validated_by,
        last_updated_on
        )
    
    for current_disease_reference in ref_list_intro:
        reference_id = insert_reference(current_disease_reference)
        my_disease_relation = create_disease_reference_relationship(get_disease_id, get_category_id,get_source_id, reference_id)


    create_disease_details(Symptoms_arr,get_disease_id, get_category_id, get_source_id,"Not Available",1)
    create_disease_details(Causes_arr,get_disease_id, get_category_id, get_source_id,"Not Available",2)
    create_disease_details(Diagnosis_arr,get_disease_id, get_category_id, get_source_id,"Not Available",3)
    create_disease_details(Management_arr,get_disease_id, get_category_id, get_source_id,"Not Available",4)
    create_disease_details(Complications_arr,get_disease_id, get_category_id, get_source_id,"Not Available",5)
    create_disease_details(Prevention_arr,get_disease_id, get_category_id, get_source_id,"Not Available",6)
    create_disease_details(Classification_arr,get_disease_id, get_category_id, get_source_id,"Not Available",7)
    create_disease_details(Transmission_arr,get_disease_id, get_category_id, get_source_id,"Not Available",8)
    create_disease_details(Treatment_arr,get_disease_id, get_category_id, get_source_id,"Not Available",9)
    create_disease_details(Cancer_Myths_arr,get_disease_id, get_category_id, get_source_id,"Not Available",10)
    create_disease_details(symptomsvoice_arr,get_disease_id, get_category_id, get_source_id,"Not Available",11)
    create_disease_details(Diseasedescription_arr,get_disease_id, get_category_id, get_source_id,"Not Available",11)


